In [6]:
import numpy as np
from sklearn.metrics import jaccard_score
from PIL import Image

gt_color_to_label = {
    (128,   0,   0) : 0,  # building
    (128, 128, 128) : 1,  # sky
    (128,  64, 128) : 2,  # road
    (128, 128,   0) : 3,  # vegetation
    (  0,   0, 192) : 4,  # sidewalk
    ( 64,   0, 128) : 5,  # car
    ( 64,  64,   0) : 6,  # pedestrain
    (  0, 128, 192) : 7,  # cyclist
    (192, 128, 128) : 8,  # signate
    ( 64,  64, 128) : 9,  # fence
    (192, 192, 128) : 10, # pole
    (  0,   0,   0) : 255 # invalid
}

img_gt_dir = '/home/ganlu/la3dm_ws/src/semantic_3d_mapping/grid_sensor/data_kitti_05/kitti_05/'
img_pred_dir = '/home/ganlu/la3dm_ws/src/semantic_3d_mapping/grid_sensor/data_kitti_05/reproj_img/'
evaluation_list = '/home/ganlu/la3dm_ws/src/semantic_3d_mapping/grid_sensor/data_kitti_05/evaluatioList.txt'
evaluation_list = np.loadtxt(evaluation_list)

img_pred_all = []
img_gt_all = []
for img_id in evaluation_list:
    img_id = np.array2string(img_id, formatter={'float_kind':lambda x: "%06i" % x})
    print(img_id)
    
    # Read images
    img_gt_color = np.array(Image.open(img_gt_dir + img_id + '.png'))
    img_pred = np.array(Image.open(img_pred_dir + img_id + '_bw.png'))

    # Convert rgb to label
    rows, cols, _ = img_gt_color.shape
    img_gt = np.zeros((rows, cols))
    for i in range(rows):
        for j in range(cols):
            img_gt[i, j] = gt_color_to_label[tuple(img_gt_color[i, j, :])]
    
    img_gt_all.append(img_gt)
    img_pred_all.append(img_pred)
    
# Flatten
img_gt_all = np.array(img_gt_all).flatten()
img_pred_all = np.array(img_pred_all).flatten()

# Ignore label 1, 6, 7, 255
for i in [1, 6, 7, 255]:
    mask = img_gt_all == i
    img_pred_all[mask] = i
    mask = img_pred_all == i
    img_gt_all[mask] = i

# IoU for each class
print( np.unique(np.concatenate((img_gt_all, img_pred_all), axis=0)) )
print( jaccard_score(img_gt_all, img_pred_all, average=None) )


000000
000020
000040
000060
000100
000120
000140
000160
000180
000200
000220
000240
000260
000280
000300
000320
000340
000360
000380
000400
000420
000440
000460
000480
000500
000520
000540
000560
000580
000600
000620
000640
000660
000680
000700
000720
000740
000760
000780
000800
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10. 255.]
[0.87517022 1.         0.93261928 0.87576128 0.81970487 0.76909557
 1.         1.         0.18563371 0.79189579 0.29612799 1.        ]


In [5]:
from sklearn.metrics import classification_report
print(classification_report(img_gt_all, img_pred_all))

              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97   1271703
         1.0       1.00      1.00      1.00    482122
         2.0       0.96      1.00      0.98   1127075
         3.0       0.92      0.98      0.95   1586253
         4.0       0.93      0.90      0.91    526878
         5.0       0.99      0.96      0.98    593776
         7.0       1.00      1.00      1.00       125
         8.0       0.98      0.27      0.43      4902
         9.0       0.97      0.81      0.88    718048
        10.0       0.89      0.55      0.68     37206
       255.0       1.00      1.00      1.00   5317312

    accuracy                           0.98  11665400
   macro avg       0.96      0.86      0.89  11665400
weighted avg       0.98      0.98      0.97  11665400

